# Test agents ⏰

## Setup

Load and/or check for needed environmental variables

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Check if key environment variables are set
env_vars = {
    "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY"),
    "ANTHROPIC_API_KEY": os.getenv("ANTHROPIC_API_KEY"),
    "LANGSMITH_API_KEY": os.getenv("LANGSMITH_API_KEY"),
    "LANGSMITH_TRACING": os.getenv("LANGSMITH_TRACING", "false"),
    "LANGSMITH_PROJECT": os.getenv("LANGSMITH_PROJECT", "default"),
}

print("Environment variables:")
for key, value in env_vars.items():
    if value and "API_KEY" in key:
        # Mask API keys for security
        print(f"  {key}: ****{value[-4:]}")
    else:
        print(f"  {key}: {value}")

# Warn if API keys are missing
if not env_vars["OPENAI_API_KEY"] and not env_vars["ANTHROPIC_API_KEY"]:
    print("\n⚠️  Warning: No API keys found. Some features may not work.")
    print("   Create a .env file with: OPENAI_API_KEY=your_key_here or ANTHROPIC_API_KEY=your_key_here")

Environment variables:
  OPENAI_API_KEY: None
  ANTHROPIC_API_KEY: ****_AAA
  LANGSMITH_API_KEY: ****7e27
  LANGSMITH_TRACING: true
  LANGSMITH_PROJECT: deep_research_from_scratch


## Load Local MCP Servers (servers/)

FastMCP-based servers in the `servers/` directory provide tools for MD simulation workflow.


In [ ]:
import sys
from pathlib import Path
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

# Get the project root directory (where servers/ is located)
project_root = Path.cwd().parent
print(f"Project root: {project_root}")

# Configure MCP client to connect to local FastMCP servers
# Each server is a Python script that implements FastMCP tools
server_config = {
    "structure": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [str(project_root / "servers" / "structure_server.py")],
    },
    "genesis": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [str(project_root / "servers" / "genesis_server.py")],
    },
    "complex": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [str(project_root / "servers" / "complex_server.py")],
    },
    "ligand": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [str(project_root / "servers" / "ligand_server.py")],
    },
    "export": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [str(project_root / "servers" / "export_server.py")],
    },
    "qc_min": {
        "transport": "stdio",
        "command": sys.executable,
        "args": [str(project_root / "servers" / "qc_min_server.py")],
    },
}

mcp_md_client = MultiServerMCPClient(server_config)

# Load tools from all MCP servers
md_tools = await mcp_md_client.get_tools()

# Display summary
num_servers = len(server_config)
print(f"\n✅ Loaded {len(md_tools)} MD workflow tools from {num_servers} servers")

# Group tools by expected server based on tool name patterns
server_tools = {
    "structure": ["fetch_pdb", "clean_structure", "add_hydrogens", "protonate_structure", 
                  "detect_modifications", "validate_structure", "create_mutated_structutre"],
    "genesis": ["boltz2_protein_from_seq", "boltz2_protein_from_fasta", "boltz2_multimer"],
    "complex": ["boltz2_complex", "boltz2_screen_ligands", "smina_dock", "refine_poses"],
    "ligand": ["smiles_to_3d", "generate_gaff_params", "create_ligand_lib", "parameterize_ligand_complete"],
    "export": ["export_amber", "export_gromacs", "export_openmm", "convert_formats", "package_files"],
    "qc_min": ["bond_check", "chirality_check", "run_full_qc", "posebusters_check", 
               "energy_minimize", "check_clashes"],
}

# Display tools by server
print("\n📦 Tools by server:")
tool_names = [t.name for t in md_tools]
for server, expected_tools in server_tools.items():
    actual_tools = [t for t in expected_tools if t in tool_names]
    print(f"\n  {server.upper()} ({len(actual_tools)} tools):")
    for tool in actual_tools:
        print(f"    • {tool}")

# Show any unexpected tools
all_expected = set(sum(server_tools.values(), []))
unexpected = [t for t in tool_names if t not in all_expected]
if unexpected:
    print(f"\n  UNEXPECTED ({len(unexpected)} tools):")
    for tool in unexpected:
        print(f"    • {tool}")

Project root: /Users/yasu/tmp/mcp-md

✅ Loaded 32 MD workflow tools from 7 servers

📦 Tools by server:

  STRUCTURE (7 tools):
    • fetch_pdb
    • clean_structure
    • add_hydrogens
    • protonate_structure
    • detect_modifications
    • validate_structure
    • create_mutated_structutre

  GENESIS (3 tools):
    • boltz2_protein_from_seq
    • boltz2_protein_from_fasta
    • boltz2_multimer

  COMPLEX (4 tools):
    • boltz2_complex
    • boltz2_screen_ligands
    • smina_dock
    • refine_poses

  LIGAND (4 tools):
    • smiles_to_3d
    • generate_gaff_params
    • create_ligand_lib
    • parameterize_ligand_complete

  ASSEMBLY (3 tools):
    • build_system_tleap
    • build_membrane_system
    • build_mixed_solvent

  EXPORT (3 tools):
    • export_amber
    • export_gromacs
    • export_openmm

  QC_MIN (4 tools):
    • bond_check
    • chirality_check
    • run_full_qc
    • posebusters_check

  UNEXPECTED (4 tools):
    • package_system
    • convert_format
    • openmm_m

## Test MD Workflow Agent

Create an agent that can use the MD workflow tools.


In [ ]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model

# Create an agent with MD workflow tools
# Model format: init_chat_model("model-name", model_provider="provider")
# Examples: 
#   - init_chat_model("claude-haiku-4-5-20251001", model_provider="anthropic")
#   - init_chat_model("gpt-4o-mini", model_provider="openai")
#   - init_chat_model("ollama:gpt-oss:20b")

#llm = ChatOpenAI(
#    model="gemma3:4b",
#    temperature=0,
#    base_url="http://localhost:11434/v1"
#)

#model = init_chat_model("gemma3:4b", model_provider="ollama")
#model = init_chat_model("ollama:gpt-oss:20b")
# Note: Using Claude 4.5 Haiku (available with your API key)
model = init_chat_model("claude-haiku-4-5-20251001", model_provider="anthropic")

md_agent = create_agent(
    model,  # モデル名（第一引数）
    md_tools,  # ツールリスト（第二引数）
    system_prompt="""You are an MD simulation preparation assistant. 
You have access to tools for:
- Structure retrieval and cleaning (PDB, AlphaFold)
- Protein structure generation from sequence (Boltz-2)
- Protein-ligand complex prediction (Boltz-2 + Smina)
- Ligand parameterization (RDKit, AmberTools)
- Format conversion and export
- Quality checks and energy minimization

When helping users, explain which tools you're using and why."""
)

 Just test say hello

In [4]:
# Minimal test
print("🧪 Testing agent response...")

result = await md_agent.ainvoke({
    "messages": [{"role": "user", "content": "Hi! What can you do?"}]
})

print("\n🤖 Agent says:")
for msg in result["messages"]:
    if msg.type == "ai" and msg.content:
        print(msg.content)

print("\n✅ Test done!")

🧪 Testing agent response...

🤖 Agent says:
Hello! I'm an AI assistant specialized in preparing molecular dynamics (MD) simulations and working with protein and ligand structures. Here's an overview of what I can help you with:

1. Structure Retrieval and Preparation
- Fetch protein structures from PDB, AlphaFold, or PDB-REDO
- Clean and validate structures
- Add hydrogens
- Detect modifications
- Check structure quality

2. Protein Structure Prediction
- Generate protein structures from amino acid sequences using Boltz-2
- Predict protein structures from FASTA files
- Create multimeric protein complex structures
- Predict protein-ligand complex structures

3. Ligand and Complex Preparation
- Generate 3D structures from SMILES strings
- Parameterize ligands for MD simulations
- Dock ligands to protein receptors
- Screen multiple ligands for binding

4. Molecular Dynamics System Setup
- Build solvated protein systems
- Create membrane protein systems
- Generate mixed solvent boxes
- Neut

Example 1: Fetch PDB structure


In [5]:
# Test: Fetch a PDB structure
print("🧪 Testing MD Agent: Fetching PDB structure 4AKE")
print("-" * 60)

try:
    result = await md_agent.ainvoke({
        "messages": [{
            "role": "user", 
            "content": "Fetch PDB structure 4AKE from the PDB database and tell me about it."
        }]
    })
    
    print("\n📝 Conversation:")
    for msg in result["messages"]:
        msg.pretty_print()
    
    print("\n✅ Test completed successfully!")
    
except Exception as e:
    print(f"\n❌ Error during test: {e}")
    import traceback
    traceback.print_exc()


🧪 Testing MD Agent: Fetching PDB structure 4AKE
------------------------------------------------------------

📝 Conversation:
================================ Human Message =================================

Fetch PDB structure 4AKE from the PDB database and tell me about it.
================================== Ai Message ==================================

[{'text': "I'll help you fetch the PDB structure for 4AKE and provide some information about it.", 'type': 'text'}, {'id': 'toolu_019NkWDdm97zHgQ34JtFdhSg', 'input': {'pdb_id': '4AKE'}, 'name': 'fetch_pdb', 'type': 'tool_use'}]
Tool Calls:
  fetch_pdb (toolu_019NkWDdm97zHgQ34JtFdhSg)
 Call ID: toolu_019NkWDdm97zHgQ34JtFdhSg
  Args:
    pdb_id: 4AKE
================================= Tool Message =================================
Name: fetch_pdb

{
  "pdb_id": "4AKE",
  "source": "pdb",
  "file_path": "output/4AKE.pdb",
  "num_atoms": 3459,
  "chains": [
    "A",
    "B"
  ]
}
================================== Ai Message =============

Example 2: List available tools

Show what tools the agent can access.


In [6]:
# Display all available tools grouped by server
print("🔧 Available MD Workflow Tools\n")

from collections import defaultdict
tools_by_server = defaultdict(list)

for tool in md_tools:
    # Tool names are prefixed with server name (e.g., "structure__fetch_pdb")
    if "__" in tool.name:
        server_name, tool_name = tool.name.split("__", 1)
        tools_by_server[server_name].append({
            "name": tool_name,
            "description": tool.description.split("\n")[0][:80]  # First line, truncated
        })

for server, tools in sorted(tools_by_server.items()):
    print(f"\n📦 {server.upper()} Server ({len(tools)} tools):")
    for tool in tools:
        print(f"   • {tool['name']}")
        print(f"     {tool['description']}")

print(f"\n📊 Total: {len(md_tools)} tools from {len(tools_by_server)} servers")


🔧 Available MD Workflow Tools


📊 Total: 32 tools from 0 servers


Example 3: Complex workflow query

Ask the agent to plan a multi-step workflow.


In [7]:
# Ask the agent to plan a workflow (without executing)
print("🧪 Testing MD Agent: Planning a protein-ligand simulation workflow")
print("-" * 70)

workflow_query = """
I want to prepare a protein-ligand complex for MD simulation. 
The protein is PDB ID 7TIM (triosephosphate isomerase) and the ligand is aspirin (SMILES: CC(=O)Oc1ccccc1C(=O)O).

Please explain what steps would be needed and which tools you would use for each step. 
Don't actually execute the tools yet, just plan the workflow.
"""

try:
    print("📤 Sending query to agent...")
    result = await md_agent.ainvoke({
        "messages": [{
            "role": "user", 
            "content": workflow_query
        }]
    })
    
    print("\n" + "=" * 70)
    print("📊 CONVERSATION ANALYSIS")
    print("=" * 70)
    
    # Count message types
    human_msgs = [m for m in result["messages"] if m.type == "human"]
    ai_msgs = [m for m in result["messages"] if m.type == "ai"]
    tool_msgs = [m for m in result["messages"] if m.type == "tool"]
    
    print(f"Total messages: {len(result['messages'])}")
    print(f"  • Human messages: {len(human_msgs)}")
    print(f"  • AI messages: {len(ai_msgs)}")
    print(f"  • Tool messages: {len(tool_msgs)}")
    
    print("\n" + "=" * 70)
    print("💬 COMPLETE CONVERSATION TRACE")
    print("=" * 70)
    
    for i, msg in enumerate(result["messages"], 1):
        print(f"\n[Message {i}] Type: {msg.type.upper()}")
        print("-" * 70)
        
        if msg.type == "human":
            print("👤 USER:")
            print(msg.content)
            
        elif msg.type == "ai":
            print("🤖 ASSISTANT:")
            
            # Show any reasoning/content
            if msg.content:
                print("\n💭 Response:")
                print(msg.content)
            
            # Show tool calls if any
            if hasattr(msg, "tool_calls") and msg.tool_calls:
                print("\n🔧 Requested tool calls:")
                for j, tool_call in enumerate(msg.tool_calls, 1):
                    print(f"  {j}. {tool_call['name']}")
                    print(f"     Args: {tool_call.get('args', {})}")
                    
        elif msg.type == "tool":
            print(f"🔨 TOOL: {msg.name}")
            result_str = str(msg.content)
            if len(result_str) > 300:
                print(f"Result: {result_str[:300]}... [truncated]")
            else:
                print(f"Result: {result_str}")
    
    print("\n" + "=" * 70)
    print("✨ FINAL WORKFLOW PLAN")
    print("=" * 70)
    
    # Extract final AI response
    final_ai_messages = [m for m in result["messages"] if m.type == "ai" and m.content]
    if final_ai_messages:
        print(final_ai_messages[-1].content)
    else:
        print("⚠️  No final plan found in conversation")
    
    print("\n" + "=" * 70)
    print("✅ Planning test completed successfully!")
    
except Exception as e:
    print(f"\n❌ Error during test: {e}")
    import traceback
    traceback.print_exc()

🧪 Testing MD Agent: Planning a protein-ligand simulation workflow
----------------------------------------------------------------------
📤 Sending query to agent...

📊 CONVERSATION ANALYSIS
Total messages: 2
  • Human messages: 1
  • AI messages: 1
  • Tool messages: 0

💬 COMPLETE CONVERSATION TRACE

[Message 1] Type: HUMAN
----------------------------------------------------------------------
👤 USER:

I want to prepare a protein-ligand complex for MD simulation. 
The protein is PDB ID 7TIM (triosephosphate isomerase) and the ligand is aspirin (SMILES: CC(=O)Oc1ccccc1C(=O)O).

Please explain what steps would be needed and which tools you would use for each step. 
Don't actually execute the tools yet, just plan the workflow.


[Message 2] Type: AI
----------------------------------------------------------------------
🤖 ASSISTANT:

💭 Response:
I'll outline a comprehensive workflow for preparing this protein-ligand complex for MD simulation, explaining the tools and rationale for each ste

## Alternative: Manual Tool Invocation

You can also call MCP tools directly without an agent. This is useful for debugging or when you need precise control.


In [12]:
# Example: Calling fetch_pdb tool directly
print("🔧 Direct Tool Invocation Example\n")

# Find the fetch_pdb tool
fetch_pdb_tool = None
for tool in md_tools:
    if "fetch_pdb" in tool.name:
        fetch_pdb_tool = tool
        break

if fetch_pdb_tool:
    print(f"📦 Tool: {fetch_pdb_tool.name}")
    print(f"📝 Description: {fetch_pdb_tool.description[:100]}...\n")
    
    try:
        # Call the tool directly with parameters
        print("Calling tool with: pdb_id='1AKE', source='pdb'")
        result = await fetch_pdb_tool.ainvoke({"pdb_id": "1AKE", "source": "pdb"})
        
        print("\n✅ Tool call successful!")
        print(f"📄 Result: {result}")
        
    except Exception as e:
        print(f"\n❌ Error calling tool: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ fetch_pdb tool not found")


🔧 Direct Tool Invocation Example

📦 Tool: fetch_pdb
📝 Description: Fetch PDB structure from RCSB PDB, AlphaFold DB, or PDB-REDO
    
    Args:
        pdb_id: PDB ID (...

Calling tool with: pdb_id='1AKE', source='pdb'

✅ Tool call successful!
📄 Result: {
  "pdb_id": "1AKE",
  "source": "pdb",
  "file_path": "output/1AKE.pdb",
  "num_atoms": 3816,
  "chains": [
    "A",
    "B"
  ]
}


## Advanced Example: ReAct Agent for Iterative Structure Prediction

Demonstrate a ReAct (Reasoning + Acting) agent that uses Boltz-2 iteratively to explore different structure prediction strategies.


In [ ]:
# Create a specialized ReAct agent for Boltz-2 structure prediction
# This agent will reason about its actions and iterate to find the best approach

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

# Filter tools to only include Boltz-2 related tools (by checking tool name)
boltz2_tools = [tool for tool in md_tools if "boltz2" in tool.name.lower()]

print(f"🔬 Available Boltz-2 tools: {len(boltz2_tools)}")
for tool in boltz2_tools:
    print(f"   • {tool.name}")

# Create ReAct agent with specialized system prompt
react_prompt = """You are a protein structure prediction specialist using Boltz-2.

Your goal is to help users predict protein structures through iterative reasoning and experimentation.

## Your Capabilities:
- boltz2_protein_from_seq: Predict structure from amino acid sequences (supports multimers)
- boltz2_protein_from_fasta: Predict structure from FASTA files
- boltz2_multimer: Predict multimeric protein complexes
- boltz2_complex: Predict protein-ligand complexes
- boltz2_screen_ligands: Screen multiple ligands against a protein

## ReAct Pattern (Reasoning + Acting):
1. **Reason**: Analyze the user's request and current situation
2. **Act**: Choose and execute the most appropriate tool
3. **Observe**: Examine the results critically
4. **Iterate**: If results are unsatisfactory, reason about improvements and try again

## Best Practices for Iteration:
- Start with the most straightforward approach
- If prediction fails, consider:
  * Sequence length (Boltz-2 works best with <1000 residues)
  * Multimer vs monomer (specify chain count if unclear)
  * Input format (sequence string vs FASTA file)
- Compare multiple predictions and explain trade-offs
- Provide clear reasoning for each decision

## Output Guidelines:
- Explain your reasoning before each action
- After each tool use, analyze the results
- If something doesn't work, explain why and propose alternatives
- Summarize findings and recommend the best approach

Remember: You can run the same tool multiple times with different parameters to explore options."""

boltz2_agent = create_agent(
    init_chat_model("claude-haiku-4-5-20251001", model_provider="anthropic"),
    boltz2_tools,
    system_prompt=react_prompt
)

print(f"\n✅ ReAct agent for Boltz-2 structure prediction created!")
print(f"   Model: claude-haiku-4-5-20251001 (Anthropic Claude 4.5 Haiku)")
print(f"   Tools: {len(boltz2_tools)} Boltz-2 tools available")
print("   This agent will reason about its actions and iterate as needed.")

🔬 Available Boltz-2 tools: 5
   • boltz2_protein_from_seq
   • boltz2_protein_from_fasta
   • boltz2_multimer
   • boltz2_complex
   • boltz2_screen_ligands

✅ ReAct agent for Boltz-2 structure prediction created!
   Model: claude-3-5-haiku-20241022 (Anthropic Claude 3.5 Haiku)
   Tools: 5 Boltz-2 tools available
   This agent will reason about its actions and iterate as needed.


### Test Case 1: Iterative Monomer vs Dimer Exploration

Ask the agent to explore whether a protein should be predicted as a monomer or dimer.


In [10]:
print("🧪 ReAct Test: Exploring oligomeric states")
print("=" * 70)

# Test query that requires iterative exploration
test_query = """
I have the sequence of Green Fluorescent Protein (GFP):
MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK

I'm not sure if GFP forms a monomer or dimer in solution. 
Can you try both approaches and help me understand which prediction might be more biologically relevant?

Please:
1. First predict it as a monomer
2. Then predict it as a dimer
3. Explain the differences you observe
4. Recommend which approach is better based on what you know about GFP
"""

try:
    print("📝 User Query:")
    print(test_query.strip())
    print("\n" + "=" * 70)
    print("🤖 Agent Response:\n")
    
    result = await boltz2_agent.ainvoke({
        "messages": [{"role": "user", "content": test_query}]
    })
    
    # Display the conversation showing ReAct pattern
    for i, msg in enumerate(result["messages"]):
        if msg.type == "human":
            print(f"\n👤 USER:")
            print(msg.content)
        elif msg.type == "ai":
            if hasattr(msg, "tool_calls") and msg.tool_calls:
                print(f"\n🤔 REASONING & ACTION {i//2}:")
                if msg.content:
                    print(f"Reasoning: {msg.content}")
                for tool_call in msg.tool_calls:
                    print(f"→ Calling: {tool_call['name']}")
                    print(f"  Args: {tool_call['args']}")
            else:
                print(f"\n🤖 FINAL RESPONSE:")
                print(msg.content)
        elif msg.type == "tool":
            print(f"\n📊 OBSERVATION:")
            print(f"Result from {msg.name}:")
            # Truncate long results
            result_str = str(msg.content)
            if len(result_str) > 300:
                print(f"{result_str[:300]}... [truncated]")
            else:
                print(result_str)
    
    print("\n" + "=" * 70)
    print("✅ ReAct test completed!")
    
except Exception as e:
    print(f"\n❌ Error during ReAct test: {e}")
    import traceback
    traceback.print_exc()


🧪 ReAct Test: Exploring oligomeric states
📝 User Query:
I have the sequence of Green Fluorescent Protein (GFP):
MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK

I'm not sure if GFP forms a monomer or dimer in solution. 
Can you try both approaches and help me understand which prediction might be more biologically relevant?

Please:
1. First predict it as a monomer
2. Then predict it as a dimer
3. Explain the differences you observe
4. Recommend which approach is better based on what you know about GFP

🤖 Agent Response:


❌ Error during ReAct test: Error calling tool 'boltz2_multimer': boltz is not available


Traceback (most recent call last):
  File "/var/folders/sq/vqh82prn4d9g1z4dx7xsdds00000gn/T/ipykernel_251/4111692077.py", line 25, in <module>
    result = await boltz2_agent.ainvoke({
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/langgraph/pregel/main.py", line 3182, in ainvoke
    async for chunk in self.astream(
  File "/Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/langgraph/pregel/main.py", line 3000, in astream
    async for _ in runner.atick(
  File "/Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/langgraph/pregel/_runner.py", line 304, in atick
    await arun_with_retry(
  File "/Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/langgraph/pregel/_retry.py", line 137, in arun_with_retry
    return await task.proc.ainvoke(task.input, config)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/l

### Test Case 2: Error Recovery and Alternative Approaches

Demonstrate how the agent handles errors and finds alternative solutions.


In [11]:
print("🧪 ReAct Test: Error recovery and adaptation")
print("=" * 70)

# Test with a challenging scenario
recovery_query = """
I want to predict the structure of a small peptide and a larger protein domain:

1. Short peptide (15 residues): MSKGEELFTGVVPIL
2. Protein domain (238 residues): MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK

Can you predict structures for both? 
If you encounter any issues (e.g., sequence too short, format problems), please explain the problem and try alternative approaches.
"""

try:
    print("📝 User Query:")
    print(recovery_query.strip())
    print("\n" + "=" * 70)
    print("🤖 Agent Working...\n")
    
    result = await boltz2_agent.ainvoke({
        "messages": [{"role": "user", "content": recovery_query}]
    })
    
    # Count reasoning-action cycles
    reasoning_cycles = sum(1 for msg in result["messages"] 
                          if msg.type == "ai" and hasattr(msg, "tool_calls") and msg.tool_calls)
    
    print(f"\n📊 Summary:")
    print(f"   Reasoning-Action cycles: {reasoning_cycles}")
    print(f"   Total messages: {len(result['messages'])}")
    
    # Show final response
    final_responses = [msg for msg in result["messages"] 
                      if msg.type == "ai" and not (hasattr(msg, "tool_calls") and msg.tool_calls)]
    if final_responses:
        print(f"\n🤖 Final Conclusion:")
        print(final_responses[-1].content)
    
    print("\n" + "=" * 70)
    print("✅ Error recovery test completed!")
    
except Exception as e:
    print(f"\n❌ Error during test: {e}")
    import traceback
    traceback.print_exc()


🧪 ReAct Test: Error recovery and adaptation
📝 User Query:
I want to predict the structure of a small peptide and a larger protein domain:

1. Short peptide (15 residues): MSKGEELFTGVVPIL
2. Protein domain (238 residues): MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK

Can you predict structures for both? 
If you encounter any issues (e.g., sequence too short, format problems), please explain the problem and try alternative approaches.

🤖 Agent Working...


❌ Error during test: Error calling tool 'boltz2_protein_from_fasta': name 'read_fasta' is not defined


Traceback (most recent call last):
  File "/var/folders/sq/vqh82prn4d9g1z4dx7xsdds00000gn/T/ipykernel_251/90240797.py", line 21, in <module>
    result = await boltz2_agent.ainvoke({
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/langgraph/pregel/main.py", line 3182, in ainvoke
    async for chunk in self.astream(
  File "/Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/langgraph/pregel/main.py", line 3000, in astream
    async for _ in runner.atick(
  File "/Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/langgraph/pregel/_runner.py", line 304, in atick
    await arun_with_retry(
  File "/Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/langgraph/pregel/_retry.py", line 137, in arun_with_retry
    return await task.proc.ainvoke(task.input, config)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yasu/miniconda3/envs/mcp-md/lib/python3.11/site-packages/lan

### Understanding the ReAct Pattern

**ReAct = Reasoning + Acting**

This pattern allows AI agents to:
1. **Think** before acting (explicit reasoning)
2. **Act** by using tools
3. **Observe** results
4. **Iterate** based on observations

**Key Advantages:**
- ✅ Transparent decision-making process
- ✅ Error recovery through iteration
- ✅ Ability to explore multiple approaches
- ✅ Learning from failures

**Example ReAct Cycle:**

```
👤 User: "Predict GFP structure as monomer and dimer"

🤔 Agent Reasoning:
   "I'll start with monomer prediction since it's simpler.
    Then I'll try dimer to compare."

🔧 Agent Action:
   → Call: boltz2_protein_from_seq(sequences=["GFP_sequence"], num_chains=1)

📊 Observation:
   "Success! Monomer structure predicted. File saved to output/"

🤔 Agent Reasoning:
   "Good, monomer worked. Now let's try dimer with 2 chains."

🔧 Agent Action:
   → Call: boltz2_protein_from_seq(sequences=["GFP_sequence"], num_chains=2)

📊 Observation:
   "Success! Dimer structure predicted. File saved to output/"

🤖 Final Response:
   "I've predicted both structures. The monomer shows..., 
    while the dimer reveals... Based on biological evidence,
    GFP typically exists as a monomer, so that prediction
    is more relevant."
```

**When to Use ReAct Agents:**
- Complex tasks requiring experimentation
- Situations with potential errors or uncertainties
- When comparing multiple approaches
- Debugging or troubleshooting workflows
